In [1]:
## Importing libraries


In [2]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,array_to_img,img_to_array
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [3]:
## Resize all the images to this

IMAGE_SIZE = [224,224]
train_path = 'datasets/train'
valid_path = 'datasets/test'

In [4]:
## Import resnet layer and add preprocessing, here we will use Imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3],weights='imagenet', include_top=False)

In [4]:
resnet.summary()   ## Do not get any images in the last block

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [5]:
## We don not train the weights.

for layer in resnet.layers:
    layer.trainable= False

In [6]:
## Useful to add output layer

folders = glob('datasets/train/*')

In [7]:
folders

['datasets/train\\Audi',
 'datasets/train\\benz',
 'datasets/train\\lamborghini',
 'datasets/train\\Maruthi']

In [8]:
x = Flatten()(resnet.output)

In [9]:
### Now you need to create the Dense layer for adding o/p

prediction = Dense(len(folders),activation='softmax')(x)

## Create a model objective 

model = Model(inputs=resnet.input,outputs= prediction)

In [20]:
model.summary()  ## Adds the layers 

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [10]:
## Now lets define loss nad optimiazer

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
## Lets generate more images by using Data Augmentaion. We only rescale for test data to be unbiased.

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True )
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
## Now make sure to give the same target size inialized for Image size

training_set = train_datagen.flow_from_directory(directory = 'datasets/train',target_size= (224,224),class_mode='categorical',batch_size=32)

Found 93 images belonging to 4 classes.


In [13]:
test_set = test_datagen.flow_from_directory(directory = 'datasets/test',target_size= (224,224),class_mode='categorical',batch_size=32)

Found 107 images belonging to 4 classes.


In [14]:
## We iull fit the model and run it

r = model.fit_generator(training_set,validation_data=test_set,epochs=50,steps_per_epoch=len(training_set),validation_steps=len(test_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
3/3 [==============================] - 24s 8s/step - loss: 8.8428 - accuracy: 0.1720 - val_loss: 3.3882 - val_accuracy: 0.2150
Epoch 2/50
3/3 [==============================] - 20s 7s/step - loss: 5.9729 - accuracy: 0.1935 - val_loss: 7.2947 - val_accuracy: 0.2150
Epoch 3/50
3/3 [==============================] - 21s 7s/step - loss: 7.6100 - accuracy: 0.2688 - val_loss: 4.3789 - val_accuracy: 0.3551
Epoch 4/50
3/3 [==============================] - 19s 6s/step - loss: 4.3845 - accuracy: 0.3441 - val_loss: 3.3750 - val_accuracy: 0.2430
Epoch 5/50
3/3 [==============================] - 20s 7s/step - loss: 3.5813 - accuracy: 0.2688 - val_loss: 2.3715 - val_accuracy: 0.4019
Epoch 6/50
3/3 [==============================] - 20s 7s/step - loss: 3.0044 - accuracy: 0.4086 - val_loss: 2.2544 - val_accuracy: 0.3738
Epoch 7/50
3/3 [==============================] - 20s 7s/step - loss: 2.3209 - accuracy: 0.3333 